In [3]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import contextimport aMRPC # aMRPC iN PYTHONPATH
from context import aMRPC
from aMRPC import utils as u
from aMRPC import datatools as dt
from aMRPC import iodata as iod
from aMRPC import wavetools as wt
from aMRPC import polytools as pt

# load data
dataframe=iod.loadEvalPoints('InputParameters.txt')

# polynomial degree
No=3
# number of resolutions
Nr=3
# threshold parameter
eps=0.01
eps=0.005
# choose sources
srcs=[0,1,2]
# format output
ocols=[0,1,2,2]
zcol=[2]
fargs={'Nr':str(Nr), 'No':str(No),'ths':str(eps)}

# approach method of pol.coefficients
method=0
# dim
dim=len(srcs)
Nrs=Nr*np.ones(dim)
#NrRange=np.arange(Nr+1)
NrRange=np.arange(Nr+1)

# generate roots, weights and also polynomials
# compare with Rooots-n-Weights
# Generate Hankel matrices
Hdict=dt.genHankel(dataframe,srcs,NrRange,No)
# Roots and Weights
R,W=dt.genRootsWeights(Hdict,method)
# (monic) orthogonal polynomial coefficients
PCdict=dt.genPCs(Hdict,method)
# normed orthogonal polynomials
nPCdict=dt.genNPCs(PCdict,R,W)

# generate wavelets and compute details
P=No+1
wv=wt.WaveTools(P)
wv.genWVlets()
# compute quantiles on roots on rescaled (0,1)-Legendre polynomials
Qdict=dt.genQuantDict(dataframe,srcs,NrRange,wv)
# details
Dtdict=dt.genDetailDict(Qdict,wv)
# stoch elements to keep (details >= threshold)
Kdict=dt.markDict4keep(Dtdict,eps)
# highest level
topKeys=dt.getTopKeys(Kdict,srcs)
# multi-keys generated from topKeys
mkArr=dt.genMkeyArr(topKeys,srcs)
# get roots and weights for the output
tR, tW=dt.getRW4mKey(mkArr,R,W)

# prepare Roots and Weights for the output
oR=u.chooseCols(tR,ocols,zcol)
oW=u.chooseCols(tW,ocols,zcol)

# gen fname and write ascii output
# roots
fname=iod.genFname(1,**fargs)
iod.writeEvalPoints(oR,fname)
# test
T=iod.loadEvalPoints(fname)
#df=abs(T-oR)
#print(df)

# weights
fname=iod.genFname(2,**fargs)
iod.writeEvalPoints(oW,fname)

# normend polynomial cfs, binary
fname=iod.genFname(7,**fargs)
iod.storeDataDict(nPCdict,fname)